In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
import pyanalib.pandas_helpers as ph
from makedf.util import *

import kinematics
import gump_cuts as gc

In [ ]:
ONBEAM = "/Users/gputnam/Work/osc/cafpyana/analysis_village/gump/gumpdev_data_onbeam.df"
OFFBEAM = "/Users/gputnam/Work/osc/cafpyana/analysis_village/gump/gumpdev_data_offbeam.df"

In [ ]:
ngates_ON = pd.read_hdf(ONBEAM, "trig_0").gate_delta.sum()
ngates_OFF = pd.read_hdf(OFFBEAM, "trig_0").gate_delta.sum()

OFF_w = ngates_ON / ngates_OFF
ngates_ON, ngates_OFF, OFF_w

In [ ]:
pd.read_hdf(ONBEAM, "bnb_0")

In [ ]:
crt_ON = pd.read_hdf(ONBEAM, "crt_0")
crt_OFF = pd.read_hdf(OFFBEAM, "crt_0")

In [ ]:
crt_ON

In [ ]:
def top_crt(crtdf):
    return (crtdf.plane >= 30) & (crtdf.plane <= 40)

In [ ]:
N,bins = np.histogram(crt_ON.time[top_crt(crt_ON)], bins=np.linspace(-10, 10, 41))
centers = (bins[:-1] + bins[:1]) / 2

plt.errorbar(centers, N, np.sqrt(N), color="black", linestyle="none", marker=".")

Noff,bins = np.histogram(crt_OFF.time[top_crt(crt_OFF)], bins=np.linspace(-10, 10, 41))
centers = (bins[:-1] + bins[:1]) / 2
plt.errorbar(centers, Noff*OFF_w, np.sqrt(Noff)*OFF_w, color="red", linestyle="none", marker=".")

In [ ]:
FILE = "/Users/gputnam/Work/osc/cafpyana/analysis_village/gump/gumpdev.df"

In [ ]:
def load_data(file, nfiles=1):
    """Load event, header, and mcnu data from HDF file."""

    for s in range(nfiles):
        print("df index:"+str(s))
        df_evt = pd.read_hdf(file, "evt_"+str(s))
        df_hdr = pd.read_hdf(file, "hdr_"+str(s))
        df_mcnu = pd.read_hdf(file, "mcnu_"+str(s))
        df_stub = pd.read_hdf(file, "stub_"+str(s))

        matchdf = df_evt.copy()
        matchdf.columns = pd.MultiIndex.from_tuples([(col, '') for col in matchdf.columns])
        df_evt = ph.multicol_merge(matchdf.reset_index(), df_mcnu.reset_index(),
                                    left_on=[("__ntuple", ""), ("entry", ""), ("tmatch_idx", "")],
                                    right_on=[("__ntuple", ""), ("entry", ""), ("rec.mc.nu..index", "")],
                                    how="left") ## -- save all sllices

        cols_to_drop = []
        for c in df_evt.columns:
            if 'GENIE' in c[0] or 'Flux' in c[0]:
                cols_to_drop.append(c)

        df_evt.drop(columns=cols_to_drop, inplace=True)
        del df_mcnu

        if s == 0:
            res_df_evt = df_evt
            res_df_hdr = df_hdr
            res_df_stub = df_stub
        else:
            res_df_evt = pd.concat([res_df_evt, df_evt])
            res_df_hdr = pd.concat([res_df_hdr, df_hdr])
            res_df_stub = pd.concat([res_df_stub, df_stub])

        del df_evt
        del df_hdr
        del df_stub

    return res_df_evt, res_df_hdr, res_df_stub

In [ ]:
df, hdr, stub = load_data(FILE)
df.columns = [c[0] for c in df.columns]

In [ ]:
ONdf = pd.read_hdf(ONBEAM, "evt_0")
OFFdf = pd.read_hdf(OFFBEAM, "evt_0")

In [ ]:
mode_list = [0, 10, 1, 2, 3]
mode_labels = ['QE', 'MEC', 'RES', 'SIS/DIS', 'COH', "other"]

def breakdown_mode(var, df):
    """Break down variable by interaction mode."""
    ret = [var[df.genie_mode == i] for i in mode_list]
    ret.append(var[sum([df.genie_mode == i for i in mode_list]) == 0])
    return ret

In [ ]:
df.columns

In [ ]:
pdg_list = [2212, 13, 211]
pdg_labels = ["$p$", "$\\mu$", "$\\pi^\\pm$", "Other"]

def breakdown_pdg(var, df, particle="p"):
    ret = [var[np.abs(df["%s_true_pdg" % particle] == i)] for i in pdg_list]
    ret.append(var[sum([np.abs(df["%s_true_pdg" % particle] == i) for i in pdg_list]) == 0])
    return ret

In [ ]:
FONTSIZE = 14
HAWKS_COLORS = ["#315031", "#d54c28", "#1e3f54", "#c89648", "#43140b", "#95af8b"]

def add_style(ax, xlabel, title="", det="ICARUS"):
    ax.tick_params(axis='both', which='both', direction='in', length=6, width=1.5, labelsize=FONTSIZE, top=True, right=True)
    for spine in ax.spines.values():
        spine.set_linewidth(1.5)
    ax.set_xlabel(xlabel, fontsize=FONTSIZE, fontweight='bold')
    ax.set_ylabel('Area Normalized', fontsize=FONTSIZE, fontweight='bold')
    ax.set_title(f"$\\bf{{{det}}}$  {title}", fontsize=FONTSIZE+2)
    ax.legend(fontsize=FONTSIZE)

In [ ]:
def cut(df):
    return gc.slcfv_cut(df, "ICARUS") & gc.pfv_cut(df, "ICARUS") & (df.crlongtrkdiry > -0.6)

In [ ]:
var = "nu_score"

pvar = breakdown_mode(df.loc[cut(df), var], df[cut(df)])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0,1,21), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, density=True)

add_style(plt.gca(), "$\\nu$ Score", "Contained")

NON,_ = np.histogram(ONdf.loc[cut(ONdf), var], bins=bins)
NOff,_ = np.histogram(OFFdf.loc[cut(OFFdf), var], bins=bins)
N = NON - NOff*OFF_w
Nerr = np.sqrt(NON + NOff*OFF_w**2)
centers = (bins[:-1] + bins[1:]) / 2
diff = (bins[1:] - bins[:-1])
norm = np.sum(N*diff)
N = N / norm
Nerr = Nerr / norm

plt.errorbar(centers, N, Nerr, color="black", linestyle="none", marker=".")

In [ ]:
var = "crlongtrkdiry"

pvar = breakdown_mode(df.loc[cut(df), var], df[cut(df)])
n, bins, _ = plt.hist(pvar, bins=np.linspace(-1,1,21), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, density=True)

add_style(plt.gca(), "CRLongTrkDirY", "Contained")

NON,_ = np.histogram(ONdf.loc[cut(ONdf), var], bins=bins)
NOff,_ = np.histogram(OFFdf.loc[cut(OFFdf), var], bins=bins)
N = NON - NOff*OFF_w
Nerr = np.sqrt(NON + NOff*OFF_w**2)
centers = (bins[:-1] + bins[1:]) / 2
diff = (bins[1:] - bins[:-1])
norm = np.sum(N*diff)
N = N / norm
Nerr = Nerr / norm

plt.errorbar(centers, N, Nerr, color="black", linestyle="none", marker=".")

In [ ]:
var = "mu_chi2_of_prot_cand"

pvar = breakdown_mode(df.loc[cut(df), var], df[cut(df)])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 60,21), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, density=True)

add_style(plt.gca(), "CRLongTrkDirY", "Contained")

NON,_ = np.histogram(ONdf.loc[cut(ONdf), var], bins=bins)
NOff,_ = np.histogram(OFFdf.loc[cut(OFFdf), var], bins=bins)
N = NON - NOff*OFF_w
Nerr = np.sqrt(NON + NOff*OFF_w**2)
centers = (bins[:-1] + bins[1:]) / 2
diff = (bins[1:] - bins[:-1])
norm = np.sum(N*diff)
N = N / norm
Nerr = Nerr / norm

plt.yscale("log")
plt.errorbar(centers, N, Nerr, color="black", linestyle="none", marker=".")

In [ ]:
var = "mu_chi2_of_prot_cand"

pvar = breakdown_mode(df.loc[cut(df), var], df[cut(df)])
n, bins, _ = plt.hist(pvar, bins=np.linspace(45, 65,11), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, density=True)

add_style(plt.gca(), "CRLongTrkDirY", "Contained")

NON,_ = np.histogram(ONdf.loc[cut(ONdf), var], bins=bins)
NOff,_ = np.histogram(OFFdf.loc[cut(OFFdf), var], bins=bins)
N = NON - NOff*OFF_w
Nerr = np.sqrt(NON + NOff*OFF_w**2)
centers = (bins[:-1] + bins[1:]) / 2
diff = (bins[1:] - bins[:-1])
norm = np.sum(N*diff)
N = N / norm
Nerr = Nerr / norm

plt.errorbar(centers, N, Nerr, color="black", linestyle="none", marker=".")

In [ ]:
var = "mu_chi2_of_prot_cand"

bins = np.linspace(45, 65,11)
pvar = breakdown_pdg(df.loc[cut(df), var], df[cut(df)])
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, label=pdg_labels,
                    color=HAWKS_COLORS[:4], density=True)

add_style(plt.gca(), "CRLongTrkDirY", "Contained")

NON,_ = np.histogram(ONdf.loc[cut(ONdf), var], bins=bins)
NOff,_ = np.histogram(OFFdf.loc[cut(OFFdf), var], bins=bins)
N = NON - NOff*OFF_w
Nerr = np.sqrt(NON + NOff*OFF_w**2)
centers = (bins[:-1] + bins[1:]) / 2
diff = (bins[1:] - bins[:-1])
norm = np.sum(N*diff)
N = N / norm
Nerr = Nerr / norm
print(NON, NOff)

plt.errorbar(centers, N, Nerr, color="black", linestyle="none", marker=".")

In [ ]:
var = "mu_chi2_of_prot_cand"

bins = np.linspace(40, 70,16)
pvar = breakdown_pdg(df.loc[cut(df), var], df[cut(df)])
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, label=pdg_labels,
                    color=HAWKS_COLORS[:4], density=True)

add_style(plt.gca(), "CRLongTrkDirY", "Contained")

NON,_ = np.histogram(ONdf.loc[cut(ONdf), var], bins=bins)
NOff,_ = np.histogram(OFFdf.loc[cut(OFFdf), var], bins=bins)
N = NON - NOff*OFF_w
Nerr = np.sqrt(NON + NOff*OFF_w**2)
centers = (bins[:-1] + bins[1:]) / 2
diff = (bins[1:] - bins[:-1])
norm = np.sum(N*diff)
N = N / norm
Nerr = Nerr / norm
print(NON, NOff)

plt.errorbar(centers, N, Nerr, color="black", linestyle="none", marker=".")